# Introduction to TensorFlow: Model saving and restoring
(Code is tailored for TF0.12 testing both type of serialization V1/V2). Thre

#### Note: 
The way serialization is carried out in TF is via *checkpoint files* (usually .ckpt). The way this is done has changed quite a lot from version to version. Most chnages were mainly adding to the previous functionality and increase scope. This was true till version 0.12(the latest) when they changed their way of doing this 
more drastically and unfortunately not compatible with previous of Tensorflow.

Prior to r0.12, the saver would produce *two* files:
```
name_of_ckpt.ckpt
name_of_ckpt.ckpt.meta
```
And a checkpoint file << checkpoint >> (one per folder of ckpt files). You would explicilty save and restore << name_of_ckpt.ckpt >>. See screenshot below (for the models saved in this tutorial)

![Image](screenshot_ckpts.png)

From r0.12, we would get, by default, *three* similar files:
```
name_of_ckpt.ckpt.data00000-of-00001
name_of_ckpt.ckpt.meta
name_of_ckpt.ckpt.index
```    

#### Revert to old version
If you are using r0.12, use: 
```
saver = tf.train.Saver(write_version = tf.train.SaverDef.V1)
```
to revert to the previous way of serializing.


#### More info:
For more information and (more advanced) functionality, check out saving and restoring documentation:
[r0.12](https://www.tensorflow.org/api_docs/python/state_ops/saving_and_restoring_variables)
[<r0.12](https://www.tensorflow.org/versions/r0.11/how_tos/variables/#saving_and_restoring)

In [1]:
# for compatibility 
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
# import tensorflow
import tensorflow as tf

In [3]:
# let create some variables
A = tf.Variable(tf.random_normal([ 3, 3]), name="A")
b = tf.Variable(tf.random_normal([ 3, 1]), name="b")
prod = tf.matmul(A,b)

# update value of b
norm = tf.sqrt(tf.reduce_sum(tf.square(prod), 0))
update_op = b.assign(prod/norm)

In [4]:
# init operation
init_op = tf.initialize_all_variables()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


## Saving a model
Now, we create a Saver with *tf.train.Saver()* to manage all variables in the model (default).

In [5]:
saver = tf.train.Saver(write_version = tf.train.SaverDef.V1)

We launch a session, where will update the value of vector b and will save the session current state at the end.

In [6]:
name_of_checkpoint = "model.ckpt"

In [7]:
with tf.Session() as sess:
    sess.run(init_op)
    
    print("Initial values:")
    print(sess.run(A))
    print(sess.run(b))
    
    print("\n Starting updating")
    for _ in range(20):
        sess.run(update_op)
        print(sess.run(b).T)
        
    print("\n Saving model to: " + name_of_checkpoint)
    saver.save(sess,name_of_checkpoint)

Initial values:
[[ 0.9529801   1.48376667 -0.01687622]
 [-0.25386065  0.50722086  1.14792073]
 [ 0.44833818 -1.62193358 -1.55676198]]
[[-0.57289195]
 [-1.19852042]
 [-1.7338351 ]]

 Starting updating
[[-0.41541019 -0.44396669  0.79393202]]
[[-0.71038347  0.52655113 -0.46701089]]
[[ 0.23975793 -0.18952386 -0.9521538 ]]
[[-0.01613209 -0.55011559  0.83493274]]
[[-0.72667694  0.58730137 -0.35639536]]
[[ 0.24644065  0.09762061 -0.96422887]]
[[ 0.21096809 -0.5966692   0.77425987]]
[[-0.7843774   0.59906703 -0.16084413]]
[[ 0.12769912  0.28212428 -0.95084107]]
[[ 0.35633397 -0.62819582  0.69166183]]
[[-0.8350063   0.53191441  0.14082462]]
[[-0.00557988  0.40412402 -0.91468722]]
[[ 0.47181052 -0.65275753  0.59270769]]
[[-0.7856524   0.34092158  0.51625836]]
[[-0.12714957  0.48771983 -0.86369115]]
[[ 0.57948017 -0.66843218  0.46626297]]
[[-0.58516878  0.0641924   0.80836672]]
[[-0.23519224  0.54792672 -0.80278319]]
[[ 0.68687892 -0.66577989  0.29143527]]
[[-0.33512041 -0.17590873  0.92560816]]


## Restoring a model
Can create a similar Saver, or even re-use the one above. I will create a new one.

In [8]:
saver2restore = tf.train.Saver(write_version = tf.train.SaverDef.V1)

Launch a session to restore and check the values. These should match the ones at the end of the session above

In [9]:
with tf.Session() as sess:
    sess.run(init_op)
    saver2restore = tf.train.import_meta_graph('./model.ckpt.meta')
    saver2restore.restore(sess, "./model.ckpt")
    
    print("Restored values:")
    print(sess.run(A))
    print(sess.run(b))

Restored values:
[[ 0.9529801   1.48376667 -0.01687622]
 [-0.25386065  0.50722086  1.14792073]
 [ 0.44833818 -1.62193358 -1.55676198]]
[[-0.33512041]
 [-0.17590873]
 [ 0.92560816]]


## Saving only certain variables/parameters rather then the whole session
This is useful, when computational graph is quite complex/large but we are interested only in some of the values -- like results of some computations. In this case, we specify when creating the Saver which variable(s) we are interested in. 

For instance in the example above, we might only be interested in the value of the computed eigenvector b.

In [10]:
saver = tf.train.Saver({"eigenv": b})
name_of_checkpoint = 'b.ckpt'

In [11]:
with tf.Session() as sess:
    sess.run(init_op)
    
    print("Initial values:")
    print(sess.run(A))
    print(sess.run(b))
    
    print("\n Starting updating")
    for _ in range(20):
        sess.run(update_op)
        print(sess.run(b).T)
        
    print("\n Saving model to: " + name_of_checkpoint)
    saver.save(sess, name_of_checkpoint)

Initial values:
[[ 0.85051274 -0.51891923  0.07480607]
 [-1.61924207  1.61457515 -1.32667363]
 [-0.33239198  1.09465504  0.72454154]]
[[ 0.25885484]
 [-0.20075834]
 [-1.38225555]]

 Starting updating
[[ 0.12869745  0.63523656 -0.76151913]]
[[-0.14971237  0.98722804  0.05447058]]
[[-0.28756171  0.79819232  0.52933669]]
[[-0.33978847  0.57737672  0.74241483]]
[[-0.36125296  0.33713183  0.86938965]]
[[-0.34923303 -0.02018894  0.93681836]]
[[-0.20203304 -0.66260403  0.72120631]]
[[ 0.13138554 -0.9881897  -0.07886076]]
[[ 0.28585932 -0.78721231 -0.54642588]]
[[ 0.34032196 -0.56221187 -0.75372332]]
[[ 0.36123201 -0.31583396 -0.87735987]]
[[ 0.34487039  0.05877826 -0.93680817]]
[[ 0.17773108  0.7186389  -0.67228693]]
[[-0.15101933  0.97947478  0.13350028]]
[[-0.29228345  0.76897275  0.5685519 ]]
[[-0.34291837  0.5443669   0.76555318]]
[[-0.36175501  0.29294074  0.8850531 ]]
[[-0.34001252 -0.0997468   0.93511611]]
[[-0.15183166 -0.77159655  0.61772645]]
[[ 0.16893971 -0.96821588 -0.1844382 ]]


## Restoring b
Please note that b gets restored correctly, while A - since it was not saved - will be re-intialize to a random value.

In [12]:
with tf.Session() as sess:
    sess.run(init_op)
    saver.restore(sess, './'+name_of_checkpoint)
    
    print("Restored values:")
    print(sess.run(A))
    print(sess.run(b))

Restored values:
[[ 0.87538993  1.65076697 -1.51639628]
 [ 0.98606044  0.90760809  0.36939907]
 [ 0.727386    0.0607335  -0.24780449]]
[[ 0.16893971]
 [-0.96821588]
 [-0.1844382 ]]
